In [1]:
library(RPostgreSQL)
library(tableone)
library(weights)
library(Matching)
library(tidyverse)

Loading required package: DBI
Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: gdata
gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith

Loading required package: mice
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
#

In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"
select hadm_id, icustay_id, echo,
    norepinephrine_max, dobutamine_flag,
    vasofreeday28, ventfreeday28,
    lactate_reduction, creatinine_reduction,
    sofa, sofa_drop_2, sofa_drop_3, day1, day2, day3,
    down2, down3,
    creatinine_diff,
    lactate_diff
from (select hadm_id, icustay_id, echo from merged_data) co
left join subgroup using (hadm_id, icustay_id, echo)
left join serum_diff using (hadm_id)
left join (select icustay_id, day1, day2, day3, down2, down3 from fluid) fl using (icustay_id)
"

In [5]:
# subgroup <- dbGetQuery(con, "select * from subgroup left join fluid using (icustay_id) left join serum_diff using (hadm_id);")
subgroup <- dbGetQuery(con, sql)
head(subgroup)

hadm_id,icustay_id,echo,norepinephrine_max,dobutamine_flag,vasofreeday28,ventfreeday28,lactate_reduction,creatinine_reduction,sofa,sofa_drop_2,sofa_drop_3,day1,day2,day3,down2,down3,creatinine_diff,lactate_diff
159895,228416,1,0.000000,0,28.00000,7.581632,0.0,1.3,3,1,0,-2110.000,-1930.000,-1475.000,-180.000,-635.000,NA,NA
169700,278148,0,0.000000,0,0.00000,0.000000,NA,0.1,8,-16,8,NA,NA,NA,NA,NA,NA,NA
103722,274174,1,0.000000,0,28.00000,28.000000,0.0,2.0,5,3,2,-300.000,-1827.000,-10.000,1527.000,-290.000,NA,NA
170515,214954,1,7.977352,0,26.04167,24.854167,1.8,0.4,8,4,5,4299.590,2899.645,-1283.263,1399.945,5582.853,0.5,NA
183493,222457,0,0.000000,0,28.00000,28.000000,NA,0.1,1,0,1,5988.072,2102.761,NA,3885.311,NA,NA,NA
134244,264413,1,2.748114,0,27.37778,24.250000,0.5,0.1,4,2,3,1507.392,-1400.000,NA,2907.392,NA,NA,NA


In [6]:
nrow(subgroup)

[1] 6361

In [7]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [8]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_lactate_first,lab_lactate_min,lab_lactate_max,lab_lactate_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1.4,1.4,1.4,0,0,1,0,0.5444486,1.836721
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2569337,1.345775
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1.6,1.6,2.6,1,0,1,0,0.3264142,3.063592
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,3.8,2.2,3.8,1,1,1,0,0.8063102,1.240217
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.2239520,1.288580
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1.4,1.4,1.4,0,0,1,0,0.3457691,2.892104


In [9]:
subgroup <- subgroup %>%
left_join(full_data %>% select(icustay_id, ps_weight, ps), by = "icustay_id") %>%
mutate(dobutamine_flag = as.factor(dobutamine_flag))
head(subgroup)

hadm_id,icustay_id,echo,norepinephrine_max,dobutamine_flag,vasofreeday28,ventfreeday28,lactate_reduction,creatinine_reduction,sofa,⋯,sofa_drop_3,day1,day2,day3,down2,down3,creatinine_diff,lactate_diff,ps_weight,ps
159895,228416,1,0.000000,0,28.00000,7.581632,0.0,1.3,3,⋯,0,-2110.000,-1930.000,-1475.000,-180.000,-635.000,NA,NA,1.836721,0.5444486
169700,278148,0,0.000000,0,0.00000,0.000000,NA,0.1,8,⋯,8,NA,NA,NA,NA,NA,NA,NA,1.345775,0.2569337
103722,274174,1,0.000000,0,28.00000,28.000000,0.0,2.0,5,⋯,2,-300.000,-1827.000,-10.000,1527.000,-290.000,NA,NA,3.063592,0.3264142
170515,214954,1,7.977352,0,26.04167,24.854167,1.8,0.4,8,⋯,5,4299.590,2899.645,-1283.263,1399.945,5582.853,0.5,NA,1.240217,0.8063102
183493,222457,0,0.000000,0,28.00000,28.000000,NA,0.1,1,⋯,1,5988.072,2102.761,NA,3885.311,NA,NA,NA,1.288580,0.2239520
134244,264413,1,2.748114,0,27.37778,24.250000,0.5,0.1,4,⋯,3,1507.392,-1400.000,NA,2907.392,NA,NA,NA,2.892104,0.3457691


In [10]:
# features <- names(subgroup) %>%
#     discard(~ .x %in% c("hadm_id", "icustay_id", "echo"))
features <- c("ventfreeday28", "vasofreeday28", "dobutamine_flag", "day1", "day2", "day3",
              "sofa_drop_2", "sofa_drop_3",
              "norepinephrine_max", "lactate_diff", "creatinine_diff")
features

[1] "ventfreeday28"      "vasofreeday28"      "dobutamine_flag"   
 [4] "day1"               "day2"               "day3"              
 [7] "sofa_drop_2"        "sofa_drop_3"        "norepinephrine_max"
[10] "lactate_diff"       "creatinine_diff"

In [11]:
tab <- CreateTableOne(vars = features, strata = "echo",
                      data = subgroup)
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [12]:
tab_df

,0,1,p,SMD
n,3099,3262,,
ventfreeday28 (mean (sd)),19.09 (13.53),18.02 (25.13),0.036,0.053
vasofreeday28 (mean (sd)),20.25 (12.61),20.13 (14.91),0.742,0.008
dobutamine_flag = 1 (%),23 (0.7),137 (4.2),<0.001,0.224
day1 (mean (sd)),1939.14 (3181.88),2456.26 (3853.10),<0.001,0.146
day2 (mean (sd)),835.26 (2429.91),1258.22 (2933.92),<0.001,0.157
day3 (mean (sd)),256.15 (2108.02),686.56 (2614.26),<0.001,0.181
sofa_drop_2 (mean (sd)),1.22 (4.34),0.89 (3.39),0.001,0.086
sofa_drop_3 (mean (sd)),2.29 (4.49),1.49 (4.07),<0.001,0.186
norepinephrine_max (mean (sd)),0.81 (2.39),1.78 (5.69),<0.001,0.221


In [13]:
ipw_svydesign <- survey::svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = subgroup)

In [14]:
tab_wtd <- svyCreateTableOne(vars = features,
                             strata = "echo",
                             data = ipw_svydesign)
capture.output(tab_wtd_df <- tab_wtd %>%
               print %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [15]:
tab_wtd_df

,0,1,p
n,5739.99,5845.39,
ventfreeday28 (mean (sd)),14.45 (15.99),19.43 (50.91),0.927
vasofreeday28 (mean (sd)),18.23 (13.77),18.95 (16.50),0.050
dobutamine_flag = 1 (%),63.2 (1.1),195.5 (3.3),<0.001
day1 (mean (sd)),3379.90 (3761.82),3185.54 (4889.07),0.001
day2 (mean (sd)),2285.72 (2701.63),1517.76 (4083.22),<0.001
day3 (mean (sd)),1113.24 (2594.41),611.07 (2818.82),<0.001
sofa_drop_2 (mean (sd)),0.91 (2.84),0.97 (2.67),<0.001
sofa_drop_3 (mean (sd)),1.18 (3.05),1.14 (3.12),<0.001
norepinephrine_max (mean (sd)),1.82 (3.31),2.44 (4.22),<0.001


In [16]:
ps_icustay_id <- data.table::fread(file.path(data_dir, "ps_icustay_id"),
                                   data.table = FALSE) %>% pull(icustay_id)

In [17]:
ps_df <- subgroup %>% filter(icustay_id %in% ps_icustay_id)

In [18]:
tab_ps <- CreateTableOne(vars = features, strata = "echo", data = ps_df)
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [19]:
tab_ps_df

,0,1,p,SMD
n,1629,1629,,
ventfreeday28 (mean (sd)),17.72 (12.46),19.80 (32.58),0.016,0.084
vasofreeday28 (mean (sd)),19.08 (12.82),21.11 (16.87),<0.001,0.136
dobutamine_flag = 1 (%),21 (1.3),34 (2.1),0.103,0.062
day1 (mean (sd)),2131.86 (3377.28),2344.77 (3747.93),0.108,0.060
day2 (mean (sd)),911.10 (2488.49),1264.00 (2870.83),0.001,0.131
day3 (mean (sd)),236.50 (2129.40),727.27 (2664.70),<0.001,0.203
sofa_drop_2 (mean (sd)),1.32 (4.52),0.71 (3.19),<0.001,0.157
sofa_drop_3 (mean (sd)),2.32 (4.88),1.13 (3.90),<0.001,0.269
norepinephrine_max (mean (sd)),1.02 (2.63),1.36 (3.16),0.001,0.118
